## Settings

### Port number, Saving

In [16]:
import os
import pandas as pd
import time
from datetime import datetime
import pyvisa
import serial
import serial.tools.list_ports
import csv

## Port number of DC Power Supply
Port_rnd = 9

## Port name of Teensy controlling Ultimaker
Port_Ultimaker_Teensy = "COM5"

## Port name of Teensy reading thermo couple
Port_thermo_Teensy = "COM7"

# Destination folder and file path
folder_path = "C:\\Users\\kary\\Documents\\9_data\\Electromagnet\\"
file_path = os.path.join(folder_path, "data.csv")

# mesh settings
x_mesh_size = 1  # mm
y_mesh_size = 1 # mm
resolution = 0.5  # mm

### DC power supply

In [2]:

#Create Resource manager instance for DC power supply
rm= pyvisa.ResourceManager()
#Check the VISA device
rm.list_resources()

('ASRL5::INSTR', 'ASRL6::INSTR', 'ASRL7::INSTR', 'ASRL9::INSTR')

In [3]:
# Start the communication by VISA identifying resource name
rnd = rm.open_resource('ASRL' + str(Port_rnd) + '::INSTR')

#Checking the machine name
print(rnd.query('*IDN?'))

RND 320-KD3305P V4.2 SN:09924475



#### Definition of Stabilized Power Supply Operating Functions

In [4]:
#Specify the communication instance, channel, voltage and current and turn on the output
def out_put_on(sps,ch,vol,cur):
    sps.write("VSET"+str(ch)+":"+str(vol)) #set ch1 output current 0.01V
    sps.write("ISET"+str(ch)+":"+str(cur)) #set ch1 output current 0.01A
    sps.write("OUT1")

def out_put_on_only_V(sps,ch,vol):
    sps.write("VSET"+str(ch)+":"+str(vol)) #set ch1 output current 0.01V
    sps.write("ISET"+str(ch)+":0") #set ch1 output current 0A
    sps.write("OUT1")

#Turn off output by specifying communication instance and channel number
def out_put_off(sps,ch):
    sps.write("VSET"+str(ch)+":0") 
    sps.write("ISET"+str(ch)+":0") 
    sps.write("OUT1")

def out_put_off_all(sps):
    sps.write("VSET1:0") 
    sps.write("ISET1:0") 
    sps.write("VSET2:0") 
    sps.write("ISET2:0") 
    sps.write("OUT1")

### Magnet meter

In [5]:

# Find and configure the serial port
def configure_serial():
    ports = serial.tools.list_ports.comports()
    for port in ports:
        if "Prolific PL2303GC USB Serial COM Port" in port.description:
            return serial.Serial(
                port.device,
                baudrate=9600,
                parity=serial.PARITY_NONE,
                stopbits=serial.STOPBITS_ONE,
                bytesize=serial.EIGHTBITS,
                timeout=1
            )
    raise IOError("Prolific PL2303GC USB Serial COM Port not found")

ser = configure_serial()

# Function to send command and check if response is #ok!
def send_command_with_ok_check(command, retries=5):
    for attempt in range(retries):
        ser.write((command + '\r\n').encode())
        print(f"Sent command: {command}")  # debug
        time.sleep(1)
        response = ser.read_all()
        print(f"Response: {response.decode('latin-1')}")  # debug
        if b'#ok!' in response:
            return True
        print(f"Retry {attempt + 1}/{retries} for command: {command}")
    return False

def send_receive_command(command):
    try:
        ser.write((command + '\r\n').encode())
        print(f"Sent command: {command}")  # debug
        time.sleep(1)  # Increase response wait time
        response = ser.read_all()  # read all responses
        return response
    except serial.SerialException as e:
        print(f"Serial exception: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None

# send data clear
if not send_command_with_ok_check("$C!"):
    print("Failed to clear data")

# setting reading interval
if not send_command_with_ok_check("$Y000001!"):
    print("Failed to set record interval")


Sent command: $C!
Response: #ok!#S000100E6011510420011!#UFFFF1806060A1D2400E6010001!#S000300E6011510420011!
Sent command: $Y000001!
Response: #ok!#UFFFF1806060A1D2500E6010001!#S000100E6011510420011!


### Control board of Ultimaker (Teensy4.0)

In [6]:
# Portrate setting
baudrate_ult = 115200

if Port_Ultimaker_Teensy is None:
    print("Selected COM port not found.")
else:
    # Make serial port instance
    try:
        ult_serialInst = serial.Serial(Port_Ultimaker_Teensy, baudrate_ult, timeout=1) 
        print("Serial port opened successfully.")
    except serial.SerialException as e:
        print(f"Error opening serial port: {e}")
        exit()

Serial port opened successfully.


### Reading Board of Thermo couple (Teensy4.0)

In [7]:
# Portrate setting
baudrate_the = 115200

if Port_thermo_Teensy is None:
    print("Selected COM port not found.")
else:
    # Make serial port instance
    try:
        thermo_seriallInst = serial.Serial(Port_thermo_Teensy, baudrate_the, timeout=1) 
        print("Serial port opened successfully.")
    except serial.SerialException as e:
        print(f"Error opening serial port: {e}")
        exit()

Serial port opened successfully.


## Adjustment of position

### Home (Please not use: Limit Switch is not working now)

In [8]:
#ult_serialInst.write(('H').encode('utf-8'))

### X-axis (Enter arbitrary distance [mm])

In [9]:
displacement = str(int(input("X axis Distance (mm)")) * 2 / 3) #adjust the number according to the result
ult_serialInst.write(('X' + displacement).encode('utf-8'))

20

### Y-axis (Enter arbitrary distance [mm])

In [10]:
displacement = str(int(input("Y axis Distance (mm)")) * 2 / 3) #adjust the number according to the result
ult_serialInst.write(('Y' + displacement).encode('utf-8'))

19

### Z-axis (Enter arbitrary distance [mm])

In [11]:
displacement = str(int(input("X axis Distance (mm)")) * 2 / 3) #adjust the number according to the result
ult_serialInst.write(('Z' + displacement).encode('utf-8'))

19

## Measurement

### DC power supply

In [14]:
#DCDC converter input active voltage
out_put_on(rnd,1,5,0.1) # channel 1, 5 V, 0.1 A

### Stepping motor move

In [18]:
x_num_points = int(x_mesh_size / resolution + 1)
y_num_points = int(y_mesh_size / resolution + 1)

# initialize data frame
data = []

def move_motor(axis, displacement):
    command = axis + str(displacement)
    ult_serialInst.write(command.encode('utf-8'))
    time.sleep(1)  # Standby time for motor to operate

def thermocouple_read():

    thermo_seriallInst.write("Reference Temperature\n".encode('utf-8'))
    time.sleep(0.1)
    ref_temp = thermo_seriallInst.readline().decode('utf-8').strip()

    thermo_seriallInst.write("Probe Temperature\n".encode('utf-8'))
    time.sleep(0.1)
    pro_temp = thermo_seriallInst.readline().decode('utf-8').strip()

    return ref_temp, pro_temp

def run_mesh_movement():
    current_x = 0.0
    current_y = 0.0
        
    for y in range(y_num_points):
        for x in range(x_num_points):
            target_y = y * resolution
            target_x = x * resolution

            # Calculate relative position on Y axis
            if target_y != current_y:
                move_motor('Y', target_y - current_y)
                current_y = target_y
            
            # Calculate relative position on X axis
            if target_x != current_x:
                move_motor('X', target_x - current_x)
                current_x = target_x

            time.sleep(1)  # wait for object to reach position
            
            reference_temp, probe_temp = thermocouple_read()

            # Read data from serial port
            data.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3], time.perf_counter(), current_x, current_y, reference_temp, probe_temp])

            # Read magnetic flux
            if not send_command_with_ok_check("$KE!"):
                print("Failed to send measurement command")

        # Reset the X-axis at the end of a row before moving to the next row
        if y != y_num_points - 1:
            move_motor('X', -current_x)
            time.sleep(1)

            reference_temp, probe_temp = thermocouple_read()

            # Read data from serial port
            data.append([datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3], time.perf_counter(), 0, current_y, reference_temp, probe_temp])
            current_x = 0.0

        # Reading magnetic flux
        if not send_command_with_ok_check("$KE!"):
            print("Failed to send measurement command")

    # Convert and save to DataFrame
    df = pd.DataFrame(data, columns=["Timestamp", "High_Precision_Timestamp", "X_Position", "Y_Position", "Reference_Temperature", "Probe_Temperature"])
    df.to_csv(file_path, index=False)
    print("Data saved to CSV")

# Perform mesh movement directly
run_mesh_movement()
ult_serialInst.close()
print("Program finished and serial port closed.")


Sent command: $KE!
Response: #S000100E6011510420011!#UFFFF1806060A1D2600E6010003!#S000300E6011510420011!#S000300E6011510420011!#UFFFF1806060A1D2700E6010003!#S000100E6011510420011!#S000300E6011510420011!#UFFFF1806060A1D2800E6010003!#S000300E6011510420011!#S000300E6011510420011!#UFFFF1806060A1D2900E6010003!#S000300E6011510420011!#S000300E6011510420011!#UFFFF1806060A1D2A00E6010003!#S000300E6011510420011!#S000100E6011510420011!#UFFFF1806060A1D2B00E6010003!#S000300E6011510420011!#S000300E6011510420011!#UFFFF1806060A1D2C00E6010003!#S000300E6011510420011!#S000300E6011510420011!#S000300E6011510420011!#UFFFF1806060A1D2D00E6010001!#S000100E6011510420011!#S000100E6011510420011!#S000100E6011510420011!#UFFFF1806060A1D2E00E6010001!#S000100E6011510420011!#S000100E6011510420011!#S000100E6011510420011!#UFFFF1806060A1D2F00E6010003!#S000300E6011510420011!#S000300E6011510420011!#S000100E6011510420011!#UFFFF1806060A1D3000E6010001!#S000100E6011510420011!#S000100E6011510420011!#S000300E6011510420011!#UFFFF18

### Voltage OFF

In [19]:
# Power off all
out_put_off_all(rnd)

## Data Save

In [20]:
# Initialize CSV file and add columns
def initialize_csv():
    if not os.path.exists(file_path):
        with open(file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Timestamp", "High_Precision_Timestamp", "X_Position", "Y_Position", "Reference_Temperature", "Probe_Temperature", "Raw_data", "Temperature_Gauss_meter", "Gauss_Value"])
    else:
        df = pd.read_csv(file_path, dtype={"Raw_data": str})
        for column in ["Raw_data", "Temperature", "Gauss_Value"]:
            if column not in df.columns:
                df[column] = ''
        df.to_csv(file_path, index=False)

# Parse response string
def parse_response(response):
    try:
        temperature = int(response[18:21], 16) / 10 if response[22:23] == '1' else int(response[18:21], 16)
        gauss_value = int(response[24:27], 16)
        if gauss_value & 0x8000:
            gauss_value -= 0x10000
        return temperature, gauss_value / 10
    except Exception as e:
        print(f"Error parsing response: {response}, error: {e}")
        return None, None

# Save magnetic flux data to CSV
def save_magnetic_flux_to_csv(response_list):
    df = pd.read_csv(file_path, dtype={"Raw_data": str})
    for i, response in enumerate(response_list):
        if i >= len(df):
            break
        temperature, gauss_value = parse_response(response)
        df.at[i, 'Raw_data'] = response
        df.at[i, 'Temperature'] = temperature
        df.at[i, 'Gauss_Value'] = gauss_value
    df.to_csv(file_path, index=False)

# Initialization and adding columns
initialize_csv()

# Send data retrieval command and get response
data_response = send_receive_command("$A!")
if data_response:
    response_list = [entry for entry in data_response.decode('latin-1').split('#') if entry.startswith('D') and entry.endswith('!')]
    save_magnetic_flux_to_csv(response_list)
    print("Data saved to CSV")
else:
    print("No data received")


Sent command: $A!
Data saved to CSV
